In [ ]:
import pandas as pd
import re
import sys
from PyPDF2 import PdfFileReader
import docx2txt
import openpyxl
import gspread
import df2gspread as d2g

In [ ]:
#Extract pdf text
file = open('pending_civil.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " " #empty variable for the extracted text
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.

In [ ]:
#regex - finds and extracts cause_numbers
finds_cause_numbers = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)#pattern for cause number
#t_causenum is today's causenumber
pending_cause_number_df = pd.DataFrame(finds_cause_numbers, columns = ['cause_number'])#puts the extract cause number into a df


In [ ]:
pending_cause_number_df

In [ ]:
gc = gspread.service_account(filename = '/Users/hector/Desktop/pau/pending_cases.json')
sht1 = gc.open_by_key('1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk')


In [ ]:
worksheet = sht1.get_worksheet(0)

In [ ]:

civil_pending_notes = pd.DataFrame(worksheet.get_all_records())


In [ ]:
#civil_pending_notes = pd.read_excel('civil_pending.xlsx')
civil_pending_notes


In [ ]:
#adds both lists together in order to search for dups later
df = civil_pending_notes.append(pd.DataFrame(pending_cause_number_df, columns=['cause_number']), ignore_index=True)
df

In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates()
df

In [ ]:
df = df.fillna(' ')
df

In [ ]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_cause_numbers.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('yesterdays_pending.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts cause numbers from the previous day's pending report
y_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
yesterday_df = pd.DataFrame(y_causenum, columns = ['cause_number'])#puts it into a df
yesterday_df.shape

In [ ]:
#adds both lists together in order to search for dups later
df = todays_df.append(pd.DataFrame(y_causenum, columns=['cause_number']), ignore_index=True)
df.shape #shape of total rows


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates()
df.shape

In [ ]:
df

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_causes.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#reads in the most recent excel file that has notes ie 'latest_df'
latest_df = pd.read_excel('pending_causes.xlsx', index_col=0)#index_col = 0 so that it does not add in another indexed col
#this will read in a google sheet that has the notes in it


In [ ]:
latest_df

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('7_pen.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts new day report cause numbers to be compared to latest report
t_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
todays_df = pd.DataFrame(t_causenum, columns = ['cause_number'])

In [ ]:
todays_df

In [ ]:
#takes the new day report and adds it to the latest report for comparison
df = latest_df.append(pd.DataFrame(todays_df, columns=['cause_number']), ignore_index=True) #adds both lists together in order to search for dups later
df


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates() 
df
###Needs to be reindexes!!

In [ ]:
#prints back to excel
#update this file
df.to_excel("_pending_causes.xlsx", sheet_name='pending_cause_numbers')  

#New version of this will go back to the google sheet

In [ ]:
#duplicated=df[df.duplicated()] #finds all duplicates
#duplicated.shape #count of rows and columns of the duplicated cause numbers

In [ ]:
#shows ONLY the duplicates
#duplicated.sort_values('cause_number')